Supply, Demand and Flow constraints

In [90]:
using JuMP, HiGHS, NamedArrays

agency = [1 2 3 4 5] # 1-Cal,2-Sorange,3-Tyran,4-Ent,5-Fard0
supply = Dict(zip( agency, [70 70 70 70 20] ))
demand = Dict(zip( agency, [40 70 20 11 15] ))
m = Model(HiGHS.Optimizer)
@variable(m, W[1:5]>=0, Bin) #Warehouse flow
@variable(m, X[1:5]>=0, Bin) #Factory open or closed
@variable(m, x[agency,agency] >= 0) #material flow from factory to warehouse
@variable(m, w[agency,agency] >= 0) #material flow from warehouse to client
@constraint(m,X[1] == 1) # Factry open at calopia
@constraint(m,W[1] == 1) # Warehouse open at calopia
@constraint(m, sup[i in agency], sum(x[i,j] for j in agency) <= supply[i]*X[i] )   # supply constraint
@expression(m, f_flow[j in agency], sum(x[i,j] for i in agency)) #factory flow
@expression(m, wh_flow[j in agency], sum(w[j,k] for k in agency)) #warehouse flow
@constraint(m,whcondition[i in agency,j in agency], w[i,j]<= 1000*W[j])
@constraint(m,con[i in 1:5],f_flow[i]-wh_flow[i] == 0) #conserve flow
@constraint(m, dem[k in agency], sum(w[j,k] for j in agency) == demand[k]) #meet demand

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5]
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 dem[1] : w[1,1] + w[2,1] + w[3,1] + w[4,1] + w[5,1] == 40.0
 dem[2] : w[1,2] + w[2,2] + w[3,2] + w[4,2] + w[5,2] == 70.0
 dem[3] : w[1,3] + w[2,3] + w[3,3] + w[4,3] + w[5,3] == 20.0
 dem[4] : w[1,4] + w[2,4] + w[3,4] + w[4,4] + w[5,4] == 11.0
 dem[5] : w[1,5] + w[2,5] + w[3,5] + w[4,5] + w[5,5] == 15.0

Flow cost

In [91]:
@expression(m, fa, sum(X[i]*4000 for i in [2,3,4])) #Opening factory cost 
@expression(m, fa_at_fard, X[5]*1500) #Opening factory cost at Fardo
@expression(m, fa_same, sum(x[i,i]*0.15/200 for i in agency)*700)
@expression(m, fa_diff, sum(x[1,j]*0.20/200 for j in [2,3,4])*700)
@expression(m, fa_fard, sum(x[i,5]*0.45/200 for i in [1,2,3,4])*700)

1.575 x[1,5] + 1.575 x[2,5] + 1.575 x[3,5] + 1.575 x[4,5]

In [92]:
@expression(m, wh, sum(W[i]*100 for i in[2,3,4,5]))
@expression(m, wh_same, sum(w[i,i]*0.15 for i in agency)*700)
@expression(m, wh_diff, sum(w[1,j]*0.20 for j in [2,3,4])*700)
@expression(m, wh_fard, sum(w[i,5]*0.40 for i in [1,2,3,4])*700)

280 w[1,5] + 280 w[2,5] + 280 w[3,5] + 280 w[4,5]

Objective function - Minimize transportation cost

In [93]:
@objective(m, Min, fa+fa_at_fard+fa_fard+fa_same+fa_diff+wh+wh_same+wh_diff+wh_fard) # minimize transportation cost
optimize!(m)

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
35 rows, 58 cols, 144 nonzeros
15 rows, 54 cols, 104 nonzeros

Solving MIP model with:
   15 rows
   54 cols (4 binary, 0 integer, 0 implied int., 50 continuous)
   104 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   400             inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   400             10037.125         96.01%        0      0      0         0     0.0s
 R       0       0         0   0.00%   6951.410714     9999.325          30.48%        0      0      0        26     0.0s
 B       0       0         0   0.00%   6951.410714     7529.075           7.67% 

Factories to be opened

In [94]:
value.(X)

5-element Vector{Float64}:
  1.0
  0.0
  1.0
 -0.0
  1.0

Warehouse to be opened

In [95]:
value.(W)

5-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0

Flow of each Factory

In [96]:
value.(x)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5]
    Dimension 2, [1, 2, 3, 4, 5]
And data, a 5×5 Matrix{Float64}:
 0.0   0.0  11.0  55.0   0.0
 0.0   0.0   0.0  -0.0   0.0
 0.0  35.0   0.0  35.0   0.0
 0.0   0.0   0.0   0.0   0.0
 0.0   5.0   0.0   0.0  15.0

Flow of each Warehouse

In [97]:
value.(w)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5]
    Dimension 2, [1, 2, 3, 4, 5]
And data, a 5×5 Matrix{Float64}:
 -0.0   0.0   0.0   0.0   0.0
 40.0   0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  11.0   0.0
 -0.0  70.0  20.0   0.0   0.0
  0.0   0.0   0.0   0.0  15.0